In [51]:
import os
import pandas as pd

In [52]:
from io import StringIO
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

In [53]:
service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
blob_container = service_client.get_container_client(os.getenv("CONTAINER_NAME"))

In [54]:
def clean_and_add_features(df: pd.DataFrame, source: str):

    df.insert(0, "Source", source)
    df["Volume"] = df["Volume"].str.replace(',','').astype(int)
    df["Date"] = pd.to_datetime(df["Date"], format='%m/%d/%Y')

    # Dates
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Year"] = df["Date"].dt.year
    df["DayOfWeek"] = df["Date"].dt.dayofweek

    df["MA_5"] = df["Close"].rolling(5).mean()
    df["MA_20"] = df["Close"].rolling(20).mean()
    df["EMA_5"] = df["Close"].ewm(span=5, adjust=False).mean()
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["STD_5"] = df["Close"].rolling(5).mean()
    df["STD_20"] = df["Close"].rolling(20).mean()

    df["DailyReturn"] = df["Close"].pct_change()
    df["Volatility"] = df["DailyReturn"].rolling(20).std()
    df["PriceChange"] = df["Close"] - df["Open"]
 
    return df

In [55]:
def upload_silver_to_blob(df, serviceClient, containerName, blobName):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)

    #blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
    blob_client = serviceClient.get_blob_client(container=containerName, blob=blobName)

    blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)


In [71]:
def main():
    print("Starting main...")
    load_dotenv()

    serviceClient = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
    blobContainer = serviceClient.get_container_client(os.getenv("CONTAINER_NAME"))

    bronzeLocation = os.getenv("BRONZE_LOCATION")
    silverFileOutput = os.getenv("SILVER_OUTPUT")
    print(bronzeLocation)
    for blob in blob_container.list_blobs(name_starts_with=f"{bronzeLocation}"):
        print(blob.name)
        blob_client = blobContainer.get_blob_client(blob.name)
        content = blob_client.download_blob().readall().decode('utf-8')

        file = pd.read_csv(StringIO(content))
        final = clean_and_add_features(file, os.path.basename(blob.name).replace('.csv',''))


        upload_silver_to_blob(final, serviceClient, blobContainer, f"{silverFileOutput}silver_output.csv")

In [72]:
main()

Starting main...
bronze/
bronze/bdo.csv


TypeError: quote_from_bytes() expected bytes

In [62]:
if __name__ == "__main__":
    main()

In [47]:
d = final

In [48]:
d

,Source,Date,Open,High,Low,Close,Volume,Month,Day,Year,DayOfWeek,MA_5,MA_20,EMA_5,EMA_20,STD_5,STD_20,DailyReturn,Volatility,PriceChange
0,bdo,2025-07-28,151.5,151.5,147.8,152.2,3012590,7,28,2025,0,NaN,NaN,152.200000,152.200000,NaN,NaN,NaN,NaN,0.7
1,bdo,2025-07-24,152.2,153.9,152.0,152.2,1307730,7,24,2025,3,NaN,NaN,152.200000,152.200000,NaN,NaN,0.000000,NaN,0.0
2,bdo,2025-07-22,149.9,149.9,148.2,149.9,2438500,7,22,2025,1,NaN,NaN,151.433333,151.980952,NaN,NaN,-0.015112,NaN,0.0
3,bdo,2025-07-21,150.9,151.0,148.9,151.0,1228600,7,21,2025,0,NaN,NaN,151.288889,151.887528,NaN,NaN,0.007338,NaN,0.1
4,bdo,2025-07-17,147.9,149.8,147.5,148.6,2047490,7,17,2025,3,150.78,NaN,150.392593,151.574430,150.78,NaN,-0.015894,NaN,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,bdo,2024-03-22,154.5,155.1,153.5,154.2,3800680,3,22,2024,4,154.40,150.530,153.685807,151.001864,154.40,150.530,0.005871,0.016249,-0.3
323,bdo,2024-03-21,154.0,155.6,154.0,154.5,3289060,3,21,2024,3,153.50,151.050,153.957204,151.335020,153.50,151.050,0.001946,0.015950,0.5
324,bdo,2024-03-20,153.4,154.8,151.8,154.0,3733010,3,20,2024,2,153.40,151.570,153.971470,151.588828,153.40,151.570,-0.003236,0.015944,0.6
325,bdo,2024-03-19,152.9,154.0,151.3,153.5,2238910,3,19,2024,1,153.90,151.905,153.814313,151.770844,153.90,151.905,-0.003247,0.015384,0.6


In [50]:
unique_sources = d['Source'].unique()
print(unique_sources)

['bdo']
